<table style="width: 100%; border-style: none">
 <tbody>
  <tr style="border-style: none">
   <td style="border-style: none; width: 1%; text-align: left; font-size: 16px">Institute for Biomedical Imaging<br />Hamburg University of Technology</td>
   <td style="border-style: none; width: 1%; font-size: 16px">&nbsp;</td>
   <td style="border-style: none; width: 1%; text-align: right; font-size: 16px">Dr. Martin Möddel<br />Marija Boberg<br />Mirco Grosser</td>
  </tr>
 </tbody>
</table>
<hr>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Computer Graphics</h1>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Exercise 2 - Ray Tracing</h1>
<h3 style="font-weight:bold; text-align: center; margin: 0px; padding:0px; margin-bottom: 20px;">summer term 2023</h3>
<hr>

* 📅 Due date: 02.05.2023, 11 a.m.
* You can earn 1 point for each task (10 points in total).
* 💣 The last task is a small coding challenge for those interested (1 bonus point).

In this exercise, you will build up a ray tracer step by step to render the following scenes: 
<table>
  <tr>
    <td><img src="Pinhole.png" width=420></td>
    <td><img src="Reflection.png" width=420></td>
  </tr>
  <tr >
    <td style="text-align: center">First scene with pinhole camera (Ex. 2)</td>
    <td style="text-align: center">Second scene with shadows and reflections (Ex. 4)</td>
  </tr>
 </table>

# 1. Basic Ray Tracer

A basic ray tracer has three main parts:
* Ray generation
* Ray intersection
* Shading

In the first task, we start with a basic ray tracer using a simple camera and basic intersection and shader functions. Later on, we will replace each part with more complex functions. 

To do so we define two types. One for a simple camera and one for a ray. The simple camera is defined by the number of pixels in x- and y-direction. The ray is defined by its origin and the pointing direction.

```julia
using StaticArrays, Plots
gr()

# Types required for basic ray generation
abstract type Camera end

mutable struct SimpleCamera <: Camera
    nx::Int
    ny::Int
end

pixelNumber(camera::Camera) = (camera.nx,camera.ny)

# Types for basic ray intersection
mutable struct Ray
    origin::SVector{3,Float32}
    direction::SVector{3,Float32}
end

origin(ray::Ray) = ray.origin
direction(ray::Ray) = ray.direction
```
*Remarks:*
* Here, we use the package `StaticArrays`, which provides a framework for implementing statically sized arrays in Julia. The speed of small `SVector`, `SMatrix` and `SArray` is often more than 10 times faster than the basic `Base.Array` type since the package provides fast implementations of common array and linear algebra operations. But for large arrays this does not hold anymore. A very rough rule of thumb is that you should consider using a normal `Array` for arrays larger than 100 elements. Whenever it is possible use `StaticArrays` for small arrays in the exercises to speed up your algorithms.
Note, that it is not possible to change a single entry of a `StaticArray` but only the array as a whole.
* Second, we use the type `Float32` instead of `Float64`. It is the standard choice in computer graphics and the computer graphics hardware is optimized for the use of `Float32`.
`Float32` values can be entered by typing for instance `1f0`. `f` replaces `e`, which is used for `Float64` values.
* For later visualization we use the package `Plots` with `GR` backend. This backend provides easy and fast plotting, but it does not rely on python like the popular backend `PyPlot`.   

In [3]:
using StaticArrays, Plots
gr()

# Types required for basic ray generation
abstract type Camera end

mutable struct SimpleCamera <: Camera
    nx::Int
    ny::Int
end

pixelNumber(camera::Camera) = (camera.nx,camera.ny)

# Types for basic ray intersection
mutable struct Ray
    origin::SVector{3,Float32}
    direction::SVector{3,Float32}
end

origin(ray::Ray) = ray.origin
direction(ray::Ray) = ray.direction

direction (generic function with 1 method)

## 1.1. Ray Generation

For ray generation complete the function 
```julia
function generateRay(camera::SimpleCamera, i::Integer, j::Integer)
    # calculate origin and direction of the ray
    
    return Ray(origin,direction)
end
``` 
such that it generates the ray $(i,j)$ of the `SimpleCamera` using the orthographic view. The `SimpleCamera` consists of an `nx` $\times$ `ny` detector pixel array with corners at $(-1,-1,1)$, $(-1,1,1)$, $(1,-1,1)$, and $(1,1,1)$. The rays point into the direction $(0,0,-1)$ and have their origin in the center of pixel $(i,j)$. `throw(DomainError)` if $(i,j)$ lies outside the valid range for this particular camera. To account for the 1-based indexing in Julia, we refer to the pixels with $i \in \{1,...,nx\}$ and $j \in \{1,...,ny\}$.

You can test your code with the following test
```julia
using Test
@testset "Ray generation" begin
  testRay1 = generateRay(SimpleCamera(500,500),252,252)
  testRay2 = generateRay(SimpleCamera(500,500),1,1)
  @test isapprox(origin(testRay1),Float32[0.006000042, -0.006000042, 1.0])
  @test isapprox(direction(testRay1),Float32[0.0, 0.0, -1.0])
  @test isapprox(origin(testRay2),Float32[-0.998, 0.998, 1.0])
  @test isapprox(direction(testRay2),Float32[0.0, 0.0, -1.0])
end;
```

In [10]:
function generateRay(camera::SimpleCamera, i::Integer, j::Integer)
    nx, ny = pixelNumber(camera)
    
    if ((i<1 || i>x || j<1 || j>y))
        throw(DomainError())
    end
    # calculate origin and direction of the ray
            
    direction = SVector{3,Float32}(0,0,-1)
    return Ray(origin,direction)
end

generateRay (generic function with 1 method)

## 1.2. Ray Intersection

Ray interesection will test if a generated ray hits any or multiple `SceneObject` inside a `Scene`. Below, the definition of these types are given. At this point we restrict ourselfs to a sphere as the only `SceneObject`, which is defined by its center and radius.
```julia
abstract type AbstractScene end
abstract type SceneObject end

mutable struct Scene <: AbstractScene
    sceneObjects::Vector{SceneObject}
end

# first SceneObject
mutable struct Sphere <: SceneObject
    center::SVector{3,Float32}
    radius::Float32
end

center(sphere::Sphere) = sphere.center
radius(sphere::Sphere) = sphere.radius
```
Using these types, we get a hierarchical structure, which we use for our intersection algorithms. First, we use the function
```julia
function intersect(ray::Ray,scene::Scene)
    isHit = false
    tHit = Inf32
    objHit = nothing
    for object in scene.sceneObjects
        hittmp,t = intersect(ray,object)
        isHit = hittmp || isHit
        if hittmp & (t < tHit) 
            tHit = t
            objHit = object
        end
    end
    # return if hit, hit point and object hit
    return isHit, tHit, objHit
end
```
that applies the intersection algorithm to each object contained in the scene. Using multiple dispatch, we can use a specific intersection algorithm optimized for each object. Here, we start with the intersection algorithm for a sphere. Later we add an axis aligned bounding box to our scene, which also requires an optimized intersection algorithm.

Write a function `intersect(ray::Ray,sphere::Sphere)`, which returns whether a ray hits a given `sphere` object and at which distance to the ray origin (parameter `t`).

*Remark:* The scalar product $\mathbf a \cdot \mathbf b$ can be calculated by `dot(a,b)`, which is included in the package `LinearAlgebra`.

You can test your code with
```julia
@testset "Ray intersection" begin
    # test scene
    sphere1 = Sphere(Float32[0.5,0,0],0.25f0)
    sphere2 = Sphere(Float32[-0.5,0,-1],0.25f0)
    scene1 = Scene(SceneObject[sphere1,sphere2])
  
    testRay1 = generateRay(SimpleCamera(500,500),252,252)
    testRay2 = generateRay(SimpleCamera(500,500),150,200)
    isHit1, tHit1, objHit1 = intersect(testRay1,scene1)
    isHit2, tHit2, objHit2 = intersect(testRay2,scene1)
    # ray 1
    @test isHit1 == false
    @test isapprox(tHit1,Inf32) 
    @test objHit1 == nothing
    # ray 2
    @test isHit2 == true
    @test isapprox(tHit2,1.8900359f0) 
    @test typeof(objHit2) == Sphere
    @test center(objHit2) == Float32[-0.5, 0.0, -1.0]
end;
```

## 1.3. Shading

Write a basic `hitShader(ray::Ray,scene::Scene)` which returns `1.0f0` if some object in the scene is hit by a ray and `0.0f0` otherwise.

Test your shader with 
```julia
@testset "Basic shader" begin
    # test scene
    sphere1 = Sphere(Float32[0.5,0,0],0.25f0)
    sphere2 = Sphere(Float32[-0.5,0,-1],0.25f0)
    scene1 = Scene(SceneObject[sphere1,sphere2])
  
    testRay1 = generateRay(SimpleCamera(500,500),252,252)
    testRay2 = generateRay(SimpleCamera(500,500),150,200)

    # test
    @test hitShader(testRay1,scene1) == 0.0f0
    @test hitShader(testRay2,scene1) == 1.0f0
end;
```

## 1.4. Put Everything Together

Put together the pieces for your first basic ray tracer in a function `tracerays(scene::AbstractScene,camera::Camera,shader::Function)` and render the follwing scene.
```julia
sphere1 = Sphere(Float32[0.5,0,0],0.25f0)
sphere2 = Sphere(Float32[-0.5,0,-1],0.25f0)
scene1 = Scene(SceneObject[sphere1,sphere2])
simpleCamera = SimpleCamera(500,500)

tracerays(scene1, simpleCamera, hitShader)
```

To visualize a frame buffer you can use the following code snippet:
```julia
screen = zeros(Float32,nx,ny)
plot(Gray.(screen'),aspect_ratio=1,framestyle=:none)
```
where `nx` $\times$ `ny` are the numbers of camera pixels.

# 2. Pinhole Camera

Now, we replace the simple camera by a pinhole camera using a perspective view.
The pinhole camera consists of the following pieces:
* An `nx` $\times$ `ny` detector pixel array with corners at $(-1,-1,1)$, $(-1,1,1)$, $(1,-1,1)$, and $(1,1,1)$.
* The eye position `eyepos`, here at $(0,0,3)$.

Define the pinhole camera by `mutable struct PinholeCamera <: Camera` and write a function `generateRay(camera::PinholeCamera, i::Int, j::Int)` to be able to render `tracerays(scene, pinholeCamera, hitShader)`.

Write a constructor `PinholeCamera(nx::Int=500,ny::Int=500)` to set up the pinhole camera and render the scene:
```julia
pinholeCamera = PinholeCamera()
tracerays(scene1, pinholeCamera, hitShader)
```
*Remark:* Have a look at https://docs.julialang.org/en/v1/manual/constructors/ to know how to create an object in Julia.

# 3. Intersection with Axis Aligned Bounding Boxes
One of the simplest geometries apart from spheres are axis aligned bounding boxes, which are oriented bounding boxes with edges aligned with the coordinate axes. They are specified by their center position and their width in x-, y-, and z-direction.

Use the type
```julia
# axis aligned bounding box
mutable struct AABB <: SceneObject
    center::SVector{3,Float32}
    # positive half length from center to face of box
    hx::Float32
    hy::Float32
    hz::Float32
end
```
and create a corresponding `intersect(ray::Ray, aabb::AABB)` function to render our second scene 
```julia
sphere = Sphere(Float32[-0.5,0.5,0.5],0.25f0)
aabb = AABB(Float32[0.1,-0.5,-0.1],0.8f0,0.25f0,0.5f0)
scene2 = Scene(SceneObject[sphere, aabb])
pinholeCamera = PinholeCamera()

tracerays(scene2, pinholeCamera, hitShader)
```

# 4. Shading
Now, we replace the simple shader by a shader with different light sources. Later on, we add shadows and reflections.

## 4.1. Normal Vectors

Most shaders require the normal vector of an object to calculate the lighting. Therefore, start by writing two functions
```julia
surfaceNormal(ray::Ray,t::Float32,sphere::Sphere)
surfaceNormal(ray::Ray,t::Float32,aabb::AABB)
```
which return the surface normal at a specific hit point $\mathbf o + t\mathbf d$. The latter can be obtained using the `ray` and the parameter `t`.

Test your functions with 
```julia
@testset "Normal vectors" begin
    # test objects
    sphere = Sphere(Float32[-0.5,0.5,0.5],0.25f0)
    aabb = AABB(Float32[0.1,-0.5,-0.1],0.8f0,0.25f0,0.5f0)
  
    testRay1 = generateRay(pinholeCamera,150,100) # intersection with sphere
    testRay2 = generateRay(pinholeCamera,150,300) # intersection with aabb

    # test
    @test surfaceNormal(testRay1,0.3972542f0,sphere) ≈ Float32[0.09167396,0.8577417,0.50584096]
    @test surfaceNormal(testRay2,0.6148762f0,aabb) ≈ Float32[0.0,0.0,1.0]
end;
```

## 4.2 Ambient Light and Lambert Lighting
Let us introduce two light sources into our rendering engine: Ambient light and sun light.
```julia
# light types
abstract type LightSource end

# collect all lights
mutable struct Lights
    sources::Vector{LightSource}
end

# ambient light
mutable struct AmbientLight <: LightSource
    # approximate intensity at scene position
    intensity::Float32
    
    AmbientLight() = new(1f0)
end

# sun light
mutable struct SunLight <: LightSource
    # approximate intensity at scene position
    intensity::Float32
    # direction of the sun rays in world coordinates
    rayDirection::SVector{3,Float32}
    
    SunLight() = new(4f0,SVector{3,Float32}(0,0,-1))
    SunLight(intensity::Float32,rayDirection::SVector{3,Float32}) = new(intensity,rayDirection)
end

intensity(light::LightSource) = light.intensity
```

Write functions `lambertLighting` for the different lights and use 
```julia
# Lambert Shader
function lambertShader(ray::Ray,scene::AbstractScene,lights::Lights)
    # intersect ray and scene
    isHit, tHit, objHit = intersect(ray,scene)
    if isHit
        # compute surface normal
        normal = surfaceNormal(ray,tHit,objHit)
        # compute lighting
        shade = lambertLighting(scene,lights,normal)
        return shade
    else
        return 0.0f0
    end
end
```
to render the scene
```julia
# initialize lights
amb = AmbientLight()
α = π*0.4
sun = SunLight(4f0,normalize(SVector{3,Float32}(0.4,-sin(α),-cos(α))))
lights = Lights([sun,amb])

tracerays(scene2, pinholeCamera, lights, lambertShader)
```
with sun light and ambient lights using diffuse objects. Again, we have a hierarchical structure. You can start with a function `lambertLighting(scene::AbstractScene,lights::Lights,normal::SVector{3,Float32})`, which applies a specific `lambertLighting` function to each light contained in `lights`. Both light sources have to be treated differently. Ambient light should always contribute to a pixel's lighting with a fixed `intensity`. In contrast, the sun light illuminates surfaces dependent on the surface normal $\mathbf n$. Let $\mathbf l$ be the unit vector pointing towards the sun, then the sun's `intensity` has to be scaled by $\text{min}(\mathbf l \cdot \mathbf n,0)$. This is refered to as Lambert lighting. Keep in mind, that a pixel's lighting can be influenced by both light sources.

Aditionally, you need a slightly adapted `tracerays(scene::AbstractScene,camera::Camera,lights::Lights,shader::Function;maxIntensity::Real=5f0)` function with two new input arguments:
* `lights`, which are passed to the shader `shader(ray,scene,lights)`
* `maxIntensity`, which determines the maximum intensity of your screen. The screen values should be rescaled such that (0,maxIntensity) is rescaled to (0,1). Afterwards all values above 1 should be set to 1 (which can be done by the function `clamp01` contained in the package `Images`).

Finally, you can test your code with
```julia
@testset "Lambert lighting" begin 
    @test lambertLighting(scene2,sun,SVector{3,Float32}(0.0,0.0,1.0)) ≈ 1.1476603f0 # sun light on the aabb
    @test lambertLighting(scene2,lights,SVector{3,Float32}(0.09167396,0.8577417,0.50584096)) ≈ 4.4740057f0 # ambient and sun light on the sphere
    @test lambertShader(generateRay(pinholeCamera,150,300),scene2,lights) ≈ 2.1476603f0 # ambient and sun light on the aabb
end;
```

## 4.3 Hard Shadows
One of the problems with the shaders used so far is that there are no shadows. This problem can be solved using the fundamental paradigm of ray tracing, by casting a ray from the object hit point towards the light source (light ray). 

For the ambient light this does not have to be done, since ambient light is unable to cast shadows by definition. However, for sun light the situation is different. If the light ray hits any object, then the objects behind it are considered to lie in shadow. In this case no light is added to this pixel.

Adapt the shader above to get a function `shadowShader(ray::Ray,scene::AbstractScene,lights::Lights)` that casts shadows for the sun light and render
```julia
tracerays(scene2, pinholeCamera, lights, shadowShader)
```
Therefore, you have to extend your lighting functions above. Don't use exactly the hit point on the object to start the light ray. Otherwise, the intersection algorithm returns the hit with the object itself.

## 4.4 Shiny Objects and Reflections
Up to now we rendered perfectly diffuse surfaces using the Lambert shader. This lighting model very well reproduces the appearance of rough surfaces (clay). However, many surfaces have some degree of shininess (polished marble, brushed metal), which leads to bright spots (highlights). Observe that in nature these highlights are centered around the point on the surface, which would reflect the light of the light source directly to the observer. Around this center the intensity usually falls off very quickly (slower for rougher surfaces). This behavior can be captured by using the Blin-Phong lighting model, which replaces $\text{min}(\mathbf l \cdot \mathbf n,0)$ by $\text{min}(\mathbf h \cdot \mathbf n,0)^p$, $p > 0$. The larger $p$ the more shiny the surface will apear. $\mathbf h$ denotes the half vector between the viewing and light-source direction. Here, use $p=8$.

Additionally, shiny surfaces will reflect some part of the light. This creates reflections of the surounding objects. Such reflections can be created by shooting yet another ray into the direction of reflection and recursively following its path, while accumulating light each time an object is hit. The reflection direction can be obtained from the direction of the incomming ray $\mathbf d$ and the normal vector of the surface by $\mathbf d - 2(\mathbf d \cdot \mathbf n)\mathbf n$. 

Adapt your lighting functions by using the Blin-Phong lighting model and partial reflections. When shading, calculate the Blin-Phong contribution first and start following the ray of reflection later. Half of the light from the reflection hit point should be added to the shading of the primary hit point. If necessary also secondary contributions should be taken into account. I.e. when the primary reflection ray hits an object, a secondary should be casted and treated just as the primary, such that the light at the secondary reflection hit point then contributes with $0.5^2$ to the light of the primary hit point.

Test your shader by rendering
```julia
tracerays(scene2, pinholeCamera, lights, reflectionShader)
```

# B1: Soft Shadows using Monte Carlo Methods

The shadows created in the last exercise had extremely sharp edges due to the fact that the sun was modeled as a point source.

To create realistic shadows we need extended light sources. For these light sources the question of occlusion can not be answered binary, since also partial occlusion might occur. Let $\mathbf p$ be the hit point of a primary ray and $T$ be the surface of an isotropic light source then the light intensity at the surface point $\mathbf p$ is given by
\begin{equation}
I_{\mathbf p} = I\int_T o(\mathbf r, \mathbf p) \text{d}^3 r,
\end{equation}
were $I$ is the intensity of the light source and $o:\mathbb R^3\times \mathbb R^3 \rightarrow \mathbb R$ is the occlusion function, which is $1$ if the path between $\mathbf p$ and $\mathbf r$ is not occluded and $0$ otherwise.

In most cases this expression can not be analytically evaluated. Therefore, we have to use numerical methods for integration. A frequently used method is the so called Monte Carlo integration, where $N$ uniform samples $\mathbf r_i \in T$ are drawn to sum up
\begin{equation}
	I_{\mathbf p} \approx I\frac{\sigma}{N}\sum_{i=1}^N o(\mathbf r_i,\mathbf p),
\end{equation}
where $\sigma$ is the size of the surface area of $T$. For a large enough number of samples this sum will be close to the integral expression above.

Use this method to create a sun, which looks like a disk from each point of view for lighting. Adapt the size of this disk and the number of drawn samples to create some visible soft shadows. Adapt your lighting functions to get a `softShadowShader` with `softShadowLighting` functions. Test your shader by rendering
```julia
tracerays(scene2, pinholeCamera, lights, softShadowShader)
```